# Employee Retention

In this challenge, you have a data set with info about the employees and have to predict when
employees are going to quit by understanding the main drivers of employee churn

As said above, the goal is to predict employee retention and understand its main drivers.
Specifically, you should:
- Assume, for each company, that the headcount starts from zero on 2011/01/23. Estimate
employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13.
That is, if by 2012/03/02 2000 people have joined company 1 and 1000 of them have
already quit, then company headcount on 2012/03/02 for company 1 would be 1000.
__You should create a table with 3 columns: day, employee_headcount, company_id.__
- What are the main factors that drive employee churn? Do they make sense? Explain your
findings.
- If you could add to this data set just one variable that could help explain employee churn,
what would that be?